In [93]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn import preprocessing
from sklearn.metrics import mean_squared_error

In [88]:
df = pd.read_excel('raw.xlsx')
df.head()

,status,card_present_flag,bpay_biller_code,account,currency,long_lat,txn_description,merchant_id,merchant_code,first_name,...,age,merchant_suburb,merchant_state,extraction,amount,transaction_id,country,customer_id,merchant_long_lat,movement
0,authorized,1.0,NaN,ACC-1598451071,AUD,153.41 -27.95,POS,81c48296-73be-44a7-befa-d053f48ce7cd,NaN,Diana,...,26,Ashmore,QLD,2018-08-01T01:01:15.000+0000,16.25,a623070bfead4541a6b0fff8a09e706c,Australia,CUS-2487424745,153.38 -27.99,debit
1,authorized,0.0,NaN,ACC-1598451071,AUD,153.41 -27.95,SALES-POS,830a451c-316e-4a6a-bf25-e37caedca49e,NaN,Diana,...,26,Sydney,NSW,2018-08-01T01:13:45.000+0000,14.19,13270a2a902145da9db4c951e04b51b9,Australia,CUS-2487424745,151.21 -33.87,debit
2,authorized,1.0,NaN,ACC-1222300524,AUD,151.23 -33.94,POS,835c231d-8cdf-4e96-859d-e9d571760cf0,NaN,Michael,...,38,Sydney,NSW,2018-08-01T01:26:15.000+0000,6.42,feb79e7ecd7048a5a36ec889d1a94270,Australia,CUS-2142601169,151.21 -33.87,debit
3,authorized,1.0,NaN,ACC-1037050564,AUD,153.10 -27.66,SALES-POS,48514682-c78a-4a88-b0da-2d6302e64673,NaN,Rhonda,...,40,Buderim,QLD,2018-08-01T01:38:45.000+0000,40.90,2698170da3704fd981b15e64a006079e,Australia,CUS-1614226872,153.05 -26.68,debit
4,authorized,1.0,NaN,ACC-1598451071,AUD,153.41 -27.95,SALES-POS,b4e02c10-0852-4273-b8fd-7b3395e32eb0,NaN,Diana,...,26,Mermaid Beach,QLD,2018-08-01T01:51:15.000+0000,3.25,329adf79878c4cf0aeb4188b4691c266,Australia,CUS-2487424745,153.44 -28.06,debit


In [100]:
customers = df['first_name'].unique()
cols = ['balance','debit','credit','name','long','lat','gender','age','salary']
df2 = pd.DataFrame(columns=cols)
for i in customers:
    a = df[df['first_name'] == i]
    temp = {'balance':0,'debit':0,'credit':0,'name':'','long':0,'lat':0,'gender':'','age':0,'salary':0}
    temp['name'] = a['first_name'].iloc[0]
    temp['gender'] = a['gender'].iloc[0]
    temp['age'] = a['age'].iloc[0]
    temp['balance'] = a['balance'].iloc[-1]
    temp['lat'] = float(a['long_lat'].iloc[0].split(' ')[1])
    temp['long'] = float(a['long_lat'].iloc[0].split(' ')[0])
    for i in range(len(a.index)):
        if a['movement'].iloc[i] == 'debit':
            temp['debit']+=a['amount'].iloc[i]
        elif a['movement'].iloc[i] == 'credit':
            temp['credit']+=a['amount'].iloc[i]
    temp['salary'] = temp['credit']*4 - temp['balance']      
    df2 = df2.append(temp, ignore_index=True)
print(df2.iloc[0])
df2 = df2.drop(columns=['name'])

balance    2134.86
debit      12020.2
credit     14191.4
name         Diana
long        153.41
lat         -27.95
gender           F
age             26
salary     54630.7
Name: 0, dtype: object


In [101]:
corr_matrix = df2.corr()
print(corr_matrix["salary"].sort_values(ascending=False))

salary     1.000000
credit     0.814241
debit      0.747932
long       0.052554
lat       -0.021344
balance   -0.569376
Name: salary, dtype: float64


In [102]:
x = df2.iloc[:,:-1]
y = df2.iloc[:,-1]
label_encoder = preprocessing.LabelEncoder()
x['gender']= label_encoder.fit_transform(x['gender'])
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.25, random_state = 0)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
print(X_train.shape,X_test.shape,y_train.shape,y_test.shape)

(60, 7) (20, 7) (60,) (20,)


In [103]:
from sklearn.linear_model import LinearRegression
regressor = LinearRegression()
regressor.fit(X_train, y_train)
y_pred = regressor.predict(X_test)

In [104]:
mean_squared_error(y_test,y_pred)

2.5358048858425616e-21